In [33]:
import csv
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.formula.api as smf
import os
from datetime import datetime

%matplotlib inline

# Functions

In [34]:
def import_dict(dict_loc):
    """ Loads a dictionary from a csv """
    df = pd.read_csv(dict_loc)
    df.index = df.iloc[:,0]
    df = df.drop(df.columns[0], axis = 1)
    return df.to_dict()

def convertDateToQuarter(date):
    quarter = (date.month-1)//3 + 1
    return (str(date.year) + 'Q' + str(quarter))

# Data Import

## Directories

In [35]:
# 605 and 606 csv directory
dir_605 = '../data/605/'
dir_606 = '../data/606/'

## Raw Data

In [36]:
## Import 605 data
# Find market center csvs
marketcenter_csv_list = [x for x in os.listdir(dir_605) if x[-4:] == '.csv']
# Get .csv directories
marketcenter_csv_dirs = [dir_605 + x for x in marketcenter_csv_list]
# Read .csv files
csv_df_list_605 = [pd.read_csv(file, sep = ',') for file in marketcenter_csv_dirs]
# Merge each marketcenter's data
rawdata_605 = pd.concat(csv_df_list_605)
# Clean up
del(csv_df_list_605)

In [37]:
## Import 606 Data

csv_df_list_606 = []
# Find broker folders
broker_folders = [x for x in os.listdir(dir_606) if '.' not in x]
# Merge .csv's for each broker
for broker in broker_folders:
    # Get file locations of csv's for each broker
    directory = dir_606 + broker
    broker_csv_list  = [x for x in os.listdir(directory)]
    broker_csv_dirs  = [dir_606 + broker + '/' + csv for csv in broker_csv_list]
    # Read csv's as dataframes
    csv_df_list_606_broker = [pd.read_csv(file) for file in broker_csv_dirs]
    csv_df_list_606.append(pd.concat(csv_df_list_606_broker))
    
# Merge each broker's data
rawdata_606 = pd.concat(csv_df_list_606)
# Clean up
del(csv_df_list_606)

## Dictionaries

In [38]:
symbol_dict = import_dict('../data/keys/symbols.csv')['Exchange']
mktctr_mpid_dict = import_dict('../data/keys/mpids.csv')['MPID']
ordertype_dict = {11: 'Market', 12: 'Limit'}
broker_vol_dict = import_dict('../data/keys/broker_volumes.csv')

# Data Prep

## Broker Data

### Prepare Raw Data

In [39]:
# Import Data
data_606 = rawdata_606.copy()

# Fix Routing Venue labels
data_606['RoutingVenue'] = data_606['RoutingVenue'].apply(
    lambda x: mktctr_mpid_dict.get(x.strip(), "(Unknown) " + str(x.strip())))

# Drop unknown routing venues
data_606 = data_606[data_606['RoutingVenue'].apply(lambda x: not x.startswith('(Unk'))]

# Convert date to quarter
data_606['Quarter'] = data_606['Date'].apply(
    lambda x: convertDateToQuarter(datetime.strptime(str(x), '%Y%m')))
data_606['Quarter'] = pd.PeriodIndex(data_606['Quarter'], freq='Q').values
data_606 = data_606.drop('Date', axis=1)

# Change column names
data_606 = data_606.rename(
    columns={'RoutingVenue': 'MarketCenter', 'Pct': 'MktShare'})

# Merge known marketcenters of same firm
data_606 = data_606.groupby(['Broker', 'Exchange', 'OrderType', 'Quarter', 'Rebate', 'MarketCenter']).sum().reset_index()

# Add binary var for presence of rebates
data_606['Rebate_Dummy'] = (data_606['Rebate'].apply(lambda x: (x > 0))
                            | data_606['Broker'].apply(lambda x: x == 'TD_Ameritrade')).apply(lambda x: int(x))


### Fill in missing 0's

In [40]:
data_606['Obs_id'] = data_606['Broker'] + '-' + data_606['MarketCenter'] + '-' + data_606['Exchange'] + '-' + data_606['OrderType']
data_606_new = data_606.copy()

dates_set = pd.Series(list(data_606['Quarter'].unique())).sort_values()

rebate_dummy_dict = {broker: data_606.query('Broker == "' + broker + '"').iloc[0]['Rebate_Dummy'] for broker in data_606['Broker'].unique()}

# from second element onwards
for quarter in dates_set.iloc[0:]: 
    
    print(quarter, end = ' ')
    mask_1 = (data_606['Quarter'] <  quarter) & (data_606['Quarter'] >= (quarter - 2)) # within last given period
    mask_2 = (data_606['Quarter'] == quarter)
    
#     print('Elements in previous quarters')
#     display(data_606.loc[mask_1].sort_values(by = 'Quarter'))
    
#     print('Elements in this quarter')
#     display(data_606.loc[mask_2])
            
    set_1 = set(data_606.loc[mask_1]['Obs_id'].unique())
    set_2 = set(data_606.loc[mask_2]['Obs_id'].unique())
    
#     display(set_2 - set_1)
    
    # missing id's (last period obs that are not in this period)
    id_list = [list(x.split('-')) for x in (set_1 - set_2)]    
#     display(id_list)
    
    # add missing id's
    print('(%d)' % len(id_list), end = ', ')
    for missing_id in id_list:
        #print(missing_id)
        
        data_606_new = data_606_new.append({'Broker': missing_id[0], 'MarketCenter': missing_id[1], 'Exchange': missing_id[2], 
                            'OrderType': missing_id[3], 'Quarter': quarter, 'Obs_id': '-'.join(missing_id),
                            'Rebate_Dummy': rebate_dummy_dict.get(missing_id[0], np.nan), 'MktShare': 0}, 
                           ignore_index = True)
    
#     print('Updated for this quarter')
#     display(data_606.loc[(data_606['Quarter'] == quarter)])

data_606 = data_606_new.copy()
data_606.head()

2010Q1 (0), 2010Q2 (4), 2010Q3 (4), 2010Q4 (16), 2011Q1 (12), 2011Q2 (4), 2011Q3 (8), 2011Q4 (20), 2012Q1 (32), 2012Q2 (28), 2012Q3 (24), 2012Q4 (20), 2013Q1 (32), 2013Q2 (36), 2013Q3 (12), 2013Q4 (16), 2014Q1 (24), 2014Q2 (144), 2014Q3 (256), 2014Q4 (240), 2015Q1 (240), 2015Q2 (364), 2015Q3 (312), 2015Q4 (280), 2016Q1 (316), 2016Q2 (532), 2016Q3 (1424), 2016Q4 (896), 2017Q1 (132), 2017Q2 (328), 2017Q3 (344), 2017Q4 (1244), 

,Broker,Exchange,OrderType,Quarter,Rebate,MarketCenter,MktShare,Rebate_Dummy,Obs_id
0,AXA,NASDAQ,Limit,2014Q1,0.0,ATDF,0.1601,0,AXA-ATDF-NASDAQ-Limit
1,AXA,NASDAQ,Limit,2014Q1,0.0,CDRG,0.1997,0,AXA-CDRG-NASDAQ-Limit
2,AXA,NASDAQ,Limit,2014Q1,0.0,FBCO,0.1205,0,AXA-FBCO-NASDAQ-Limit
3,AXA,NASDAQ,Limit,2014Q1,0.0,G1ES,0.0293,0,AXA-G1ES-NASDAQ-Limit
4,AXA,NASDAQ,Limit,2014Q1,0.0,KCG,0.4148,0,AXA-KCG-NASDAQ-Limit


## Market Center Data

In [41]:
# Import data
data_605 = rawdata_605.copy()

# Quarter column
data_605['Quarter'] = data_605['idate'].apply(lambda x: convertDateToQuarter(datetime.strptime(str(x), '%Y%m')))
data_605['Quarter'] = pd.PeriodIndex(data_605['Quarter'], freq='Q').values
data_605 = data_605.drop('idate', axis = 1)

# Temporary Variables for Aggregation
data_605['PrImp_TotalT']     = data_605['PrImpShares']    * data_605['PrImp_AvgT']
data_605['PrImp_TotalAmt']   = data_605['PrImpShares']    * data_605['PrImp_AvgAmt']
data_605['ATQ_TotalT']       = data_605['ATQShares']      * data_605['ATQ_AvgT']
data_605['OTQ_TotalT']       = data_605['OTQShares']      * data_605['OTQ_AvgT']
data_605['AvgRealSpread_T']  = data_605['AvgRealSpread']  * data_605['ExecShares']
data_605['AvgEffecSpread_T'] = data_605['AvgEffecSpread'] * data_605['ExecShares']

data_605 = data_605.groupby(['MarketCenter', 'Quarter', 'Exchange', 'OrderCode']) \
        .sum().reset_index()

# Reconstruct original variables
data_605['PrImp_AvgT']     = data_605['PrImp_TotalT']     / data_605['PrImpShares']
data_605['PrImp_AvgAmt']   = data_605['PrImp_TotalAmt']   / data_605['PrImpShares'] 
data_605['ATQ_AvgT']       = data_605['ATQ_TotalT']       / data_605['ATQShares']
data_605['OTQ_AvgT']       = data_605['OTQ_TotalT']       / data_605['OTQShares']
data_605['AvgRealSpread']  = data_605['AvgRealSpread_T']  / data_605['ExecShares']
data_605['AvgEffecSpread'] = data_605['AvgEffecSpread_T'] / data_605['ExecShares'] 
data_605['PrImp_Pct']      = data_605['PrImpShares']      / data_605['ExecShares']
data_605['ATQ_Pct']        = data_605['ATQShares']        / data_605['ExecShares']
data_605['OTQ_Pct']        = data_605['OTQShares']        / data_605['ExecShares']

## New Vars

# Absolute
data_605['OrderType']    = data_605['OrderCode'].apply(lambda x: ordertype_dict.get(x, 'Other'))
data_605['PrImp_ExpAmt'] = data_605['PrImp_AvgAmt'] * data_605['PrImp_Pct']
data_605['All_AvgT']     = (data_605['PrImp_TotalT'] + data_605['ATQ_TotalT'] + data_605['OTQ_TotalT']) \
                            / data_605['ExecShares']
data_605 = data_605.rename(columns = {'idate': 'Date'})

# Relative values
data_605_grouped = data_605.groupby(['Exchange', 'OrderType', 'Quarter'])

data_605['MktCtrAvg_PrImp_Pct']  = data_605_grouped['PrImp_Pct'].transform("mean")
data_605['Rel_PrImp_Pct']        = data_605['PrImp_Pct'] - data_605['MktCtrAvg_PrImp_Pct']
data_605['MktCtrAvg_PrImp_AvgT'] = data_605_grouped['PrImp_AvgT'].transform("mean")
data_605['Rel_PrImp_AvgT']       = data_605['PrImp_AvgT'] - data_605['MktCtrAvg_PrImp_AvgT']    
data_605['MktCtrAvg_PrImp_ExpAmt'] = data_605_grouped['PrImp_ExpAmt'].transform("mean")
data_605['Rel_PrImp_ExpAmt']       = data_605['PrImp_ExpAmt'] - data_605['MktCtrAvg_PrImp_ExpAmt']   
data_605['MktCtrAvg_All_AvgT'] = data_605_grouped['All_AvgT'].transform("mean")
data_605['Rel_All_AvgT']       = data_605['All_AvgT'] - data_605['MktCtrAvg_All_AvgT']   

data_605.head()

,MarketCenter,Quarter,Exchange,OrderCode,CoveredOrders,CoveredShares,CancelledShares,MktCtrExecShares,AwayExecShares,ExecShares_0_9,...,PrImp_ExpAmt,All_AvgT,MktCtrAvg_PrImp_Pct,Rel_PrImp_Pct,MktCtrAvg_PrImp_AvgT,Rel_PrImp_AvgT,MktCtrAvg_PrImp_ExpAmt,Rel_PrImp_ExpAmt,MktCtrAvg_All_AvgT,Rel_All_AvgT
0,AQUA,2010Q1,NASDAQ,11,72,367000,113800,250900,0,250900,...,0.010296,0.065564,0.557361,0.442639,0.357667,-0.292103,0.003442,0.006854,0.714655,-0.649091
1,AQUA,2010Q1,NASDAQ,12,43168,172330609,171680431,321000,49778,227200,...,0.009834,205.496942,0.405479,0.542231,34.086084,132.742186,0.002626,0.007208,84.459308,121.037634
2,AQUA,2010Q1,NASDAQ,13,20391,68389031,68061331,110900,9700,73000,...,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000
3,AQUA,2010Q1,NASDAQ,14,13031,47528964,47400706,0,8158,0,...,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000
4,AQUA,2010Q1,NASDAQ,15,19537,74494919,74369968,0,9869,0,...,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000


# Merge Datasets

In [42]:
data_merged = data_605.merge(data_606)

data_merged = data_merged.query('OrderCode < 13')
data_merged = data_merged.drop('Obs_id', axis = 1)
data_merged.set_index(['Quarter'])
data_merged['Broker_Size'] = data_merged['Broker'].apply(lambda x: broker_vol_dict['Size'].get(x))


print('Total Observations: ' + str(len(data_merged)))
print('Brokers: ' + str(len(set(list(data_merged['Broker'])))))
print('Market Centers: ' + str(len(set(list(data_merged['MarketCenter'])))))

data_merged.head()

Total Observations: 8318
Brokers: 40
Market Centers: 12


,MarketCenter,Quarter,Exchange,OrderCode,CoveredOrders,CoveredShares,CancelledShares,MktCtrExecShares,AwayExecShares,ExecShares_0_9,...,Rel_PrImp_AvgT,MktCtrAvg_PrImp_ExpAmt,Rel_PrImp_ExpAmt,MktCtrAvg_All_AvgT,Rel_All_AvgT,Broker,Rebate,MktShare,Rebate_Dummy,Broker_Size
0,AQUA,2015Q4,Other,11,1,5000,5000,0,0,0,...,NaN,0.004768,NaN,20.208101,NaN,Deutsche,1.0,0.0,1,2.045009e+10
1,AQUA,2015Q4,Other,12,96,375200,223200,0,0,0,...,NaN,0.001124,NaN,19.623809,NaN,Deutsche,1.0,0.0,1,2.045009e+10
5,AQUA,2016Q1,Other,11,1,3000,3000,0,0,0,...,NaN,0.006184,NaN,21.218073,NaN,Deutsche,NaN,0.0,1,2.045009e+10
6,AQUA,2016Q1,Other,12,108,457600,202800,0,0,0,...,NaN,0.001188,NaN,15.650414,NaN,Deutsche,NaN,0.0,1,2.045009e+10
10,AQUA,2016Q2,NASDAQ,11,92,465340,158940,303400,0,296000,...,0.313173,0.010230,0.0196,0.200564,0.300425,Credit Suisse,1.0,0.0,1,3.266721e+10


# Data Export

## Panel

In [43]:
# test
data_merged[data_merged['Quarter'] == pd.Period('2017Q1')].query('Broker == "Deutsche" & OrderType == "Market" & Exchange == "NASDAQ"')

,MarketCenter,Quarter,Exchange,OrderCode,CoveredOrders,CoveredShares,CancelledShares,MktCtrExecShares,AwayExecShares,ExecShares_0_9,...,Rel_PrImp_AvgT,MktCtrAvg_PrImp_ExpAmt,Rel_PrImp_ExpAmt,MktCtrAvg_All_AvgT,Rel_All_AvgT,Broker,Rebate,MktShare,Rebate_Dummy,Broker_Size
693,ARCA,2017Q1,NASDAQ,11,104475,61806548,19612,36313895,25472391,61785886,...,-0.113232,0.006163,-0.004331,0.211129,-0.208189,Deutsche,1.0,0.0002,1,2.045009e+10
6781,CITI,2017Q1,NASDAQ,11,6,11095,9802,0,0,0,...,NaN,0.006163,NaN,0.211129,NaN,Deutsche,1.0,0.0000,1,2.045009e+10
7048,EDGX,2017Q1,NASDAQ,11,60462,37444426,29307707,5697480,2439239,8136719,...,-0.118755,0.006163,-0.003969,0.211129,-0.207215,Deutsche,1.0,0.0000,1,2.045009e+10
10003,FBCO,2017Q1,NASDAQ,11,15092,15678915,26687,145637,15506591,15604759,...,0.088561,0.006163,-0.003845,0.211129,0.099879,Deutsche,1.0,0.0000,1,2.045009e+10
18696,UBSS,2017Q1,NASDAQ,11,940822,698972967,845336,469982092,223946827,693152926,...,0.142654,0.006163,0.002769,0.211129,0.462117,Deutsche,1.0,0.0016,1,2.045009e+10


## First Difference

In [44]:
def tempfunc(data, broker, marketcenter, exchange, ordertype):
    
    data = data.drop('Rebate_Dummy', axis = 1)
    data['Broker'] = broker
    data['MarketCenter'] = marketcenter
    data['Exchange'] = exchange
    data['OrderType'] = ordertype
    data['Rebate_Dummy'] = data_606.query(query).iloc[0]['Rebate_Dummy']
    
    return data

def getDifferencedData(data_merged, query_command):
    
    if query_command != "":
        data_merged = data_merged.query(query_command)

    data_merged_diff = data_merged.groupby(['Broker', 'MarketCenter', 'Exchange', 'OrderType']).diff(1).dropna()
    
    return data_merged_diff

data_merged_diff_list = []
data_merged_diff_lag_list = []

brokers = list(set(list(data_merged['Broker'])))
marketcenters = list(set(list(data_merged['MarketCenter'])))
exchanges = list(set(list(data_merged['Exchange'])))
ordertypes = ['Market','Limit']

for broker in brokers:
    for marketcenter in marketcenters:
        for exchange in exchanges:
            for ordertype in ordertypes:
                query = "Broker == '" + broker + "' & OrderType == '" + ordertype \
                + "' & Exchange == '" + exchange + "' & MarketCenter == '" + marketcenter + "'"
                data_merged_diff_temp = getDifferencedData(data_merged, query)
                #data_merged_diff_lag_temp = getDifferencedData(data_merged_lag, query)
                
                if len(data_merged_diff_temp) > 0:
                    data_merged_diff_temp = tempfunc(data_merged_diff_temp, broker, marketcenter, 
                                                     exchange, ordertype)
                    #data_merged_diff_lag_temp = tempfunc(data_merged_diff_lag_temp, broker, marketcenter, 
                    #                                 exchange, ordertype)
                    
                    data_merged_diff_list.append(data_merged_diff_temp)
                    #data_merged_diff_lag_list.append(data_merged_diff_lag_temp)

## With Binaries

In [127]:
data_clean = data_merged.dropna().copy()

# {Market Center, Broker, Order Type, Exchange} dummies
temp_dict = {}

for mktctr in list(set(list(data_clean['MarketCenter'])))[:-1]:
    
    print('Processing: ' + mktctr)
    
    column_label = (mktctr + '_ind')
    data_clean[column_label] = data_clean['MarketCenter'].apply(lambda x: int(x == mktctr))
    
    # query only brokers that send to this market center
    query_txt = 'MarketCenter == "' + mktctr + '"'
    
    # all brokers except the last to prevent multicollinearity
    for brk in list(set(list(data_clean.query(query_txt)['Broker'])))[:-1]: 
        
        # Add column
        column_label = mktctr + '_' + brk.replace(' ', '_') + '_ind'
        data_clean[column_label] = data_clean[mktctr + '_ind'] & data_clean['Broker'].apply(lambda x: int(x == brk))
        
        query_txt2 = 'MarketCenter == "' + mktctr + '" & Broker == "' + brk + '"'
        
        for exc in list(set(list(data_clean.query(query_txt2)['Exchange'])))[:-1]: 
            
            # Add column
            column_label = mktctr + '_' + brk.replace(' ', '_') + '_' + exc + '_ind'
            data_clean[column_label] = data_clean[mktctr + '_ind'] & data_clean['Broker'].apply(lambda x: int(x == brk)) & \
                data_clean['Exchange'].apply(lambda x: int(x == exc))
                
            query_txt3 = 'MarketCenter == "' + mktctr + '" & Broker == "' + brk + '" & Exchange == "' + exc + '"'    
                
            for ot in list(set(list(data_clean.query(query_txt3)['OrderType'])))[:-1]:     
                
                # Add column
                column_label = mktctr + '_' + brk.replace(' ', '_') + '_' + exc + '_' + ot + '_ind'
                data_clean[column_label] = data_clean[mktctr + '_ind'] & data_clean['Broker'].apply(lambda x: int(x == brk)) & \
                    data_clean['Exchange'].apply(lambda x: int(x == exc)) & data_clean['OrderType'].apply(lambda x: int(x == ot))
        
        
print('Complete')

Processing: ARCA
Processing: VRTU
Processing: BNYC
Processing: UBSS
Processing: AQUA
Processing: CDRG
Processing: G1ES
Processing: EDGX
Processing: CITI
Complete


## Fixed Effects

In [45]:
data_merged_demeaned = data_merged.dropna() - data_merged.dropna().groupby(
    ['Broker', 'MarketCenter', 'Exchange', 'OrderType']).transform("mean")

data_merged_demeaned[['Broker', 'Exchange', 'MarketCenter', 'OrderType', 'Quarter']] = data_merged.dropna()[['Broker', 'Exchange', 'MarketCenter', 'OrderType', 'Quarter']]

data_merged_demeaned['Rebate_Dummy'] = data_merged.dropna()['Rebate_Dummy']

## To CSV

In [47]:
# Panel
data_merged.to_csv('../data/processed/regression_data_levels.csv', index=False)

# Differenced
pd.concat(data_merged_diff_list).to_csv(
    '../data/processed/regression_data_fdiffs.csv', index=False)

# With Binary Vars
data_clean.to_csv(
    '../data/processed/regression_data_levels_binaries.csv', index=False)

# Demeaned
data_merged_demeaned.to_csv(
    '../data/processed/regression_data_levels_demeaned.csv', index=False)

# 605 and 606
data_605.to_csv('../data/processed/605_processed.csv')
data_606.to_csv('../data/processed/606_processed.csv')